In [1]:
%load_ext autoreload
%autoreload 2

from zxfermion.types import Node, GadgetLeg, Gadget, GadgetCircuit
from openfermion import FermionOperator, QubitOperator, jordan_wigner

import pyscf
import pyzx as zx
import numpy as np
import openfermionpyscf

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# ansatze with same operator set

linear_ansatze = [Ansatz(result_id=f'{i}.{j}', geometry='H4_linear')
                  for i in range(1, 9)
                  for j in range(1, 11)]

grouped_ansatze = {}
for ansatz in linear_ansatze:
    ansatz_set = frozenset(ansatz.operator_order)
    if ansatz_set in grouped_ansatze:
        grouped_ansatze[ansatz_set].append(ansatz)
    else:
        grouped_ansatze[ansatz_set] = [ansatz]

grouped_ansatze = list(grouped_ansatze.values())
grouped_ansatze.sort(reverse=True, key=lambda group: len(group))

print(f'Total ansatze number: {len(linear_ansatze)}')
print(f'Unique ansatze number: {len(grouped_ansatze)}')
print(f'Largest group size: {len(grouped_ansatze[0])}', '\n')

for idx, group in enumerate(grouped_ansatze):
    unique_ansatze = len(set([tuple(ansatz.operator_order) for ansatz in group]))
    print(f'Group: {idx+1}    Unique Ansatze: {unique_ansatze}')
    for ansatz in group:
        print(f'Result ID: {ansatz.result_id}    Operator Order: {ansatz.operator_order}')
    print()

Total ansatze number: 80
Unique ansatze number: 30
Largest group size: 18 

Group: 1    Unique Ansatze: 15
Result ID: 1.1    Operator Order: [8, 7, 6, 12, 9, 1, 7, 2, 5, 11, 10, 2, 5, 11]
Result ID: 1.3    Operator Order: [10, 1, 7, 12, 8, 6, 5, 2, 7, 11, 9, 5, 10, 11]
Result ID: 1.4    Operator Order: [9, 6, 10, 1, 6, 10, 9, 5, 2, 7, 11, 9, 8, 12]
Result ID: 1.6    Operator Order: [10, 6, 9, 6, 1, 7, 9, 8, 12, 2, 5, 10, 9, 11]
Result ID: 2.2    Operator Order: [9, 10, 1, 6, 2, 11, 8, 2, 5, 7, 12, 2, 8, 9]
Result ID: 2.5    Operator Order: [10, 6, 7, 11, 1, 5, 9, 12, 2, 9, 8, 2, 7, 8]
Result ID: 2.6    Operator Order: [9, 6, 10, 1, 6, 10, 9, 5, 2, 7, 11, 9, 8, 12]
Result ID: 3.1    Operator Order: [8, 6, 7, 10, 1, 11, 10, 2, 8, 9, 5, 11, 12, 10]
Result ID: 3.3    Operator Order: [8, 6, 7, 10, 1, 11, 10, 2, 8, 9, 5, 11, 12, 10]
Result ID: 3.10    Operator Order: [10, 7, 6, 11, 1, 5, 7, 12, 8, 2, 11, 7, 9, 2]
Result ID: 4.5    Operator Order: [11, 1, 9, 1, 6, 12, 8, 5, 12, 2, 5, 10, 7, 1

In [6]:
# ansatze with same operator set and same frequency of operators

class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(self.items()))

linear_ansatze = [Ansatz(result_id=f'{i}.{j}', geometry='H4_linear')
                  for i in range(1, 9)
                  for j in range(1, 11)]

grouped_ansatze = {}
for ansatz in linear_ansatze:
    operator_freq = hashabledict({operator: ansatz.operators.count(operator) for operator in ansatz.operators})
    if operator_freq in grouped_ansatze:
        grouped_ansatze[operator_freq].append(ansatz)
    else:
        grouped_ansatze[operator_freq] = [ansatz]

grouped_ansatze = list(grouped_ansatze.values())
grouped_ansatze.sort(reverse=True, key=lambda group: len(group))


print(f'Total ansatze number: {len(linear_ansatze)}')
print(f'Unique ansatze number: {len(grouped_ansatze)}')
print(f'Largest group size: {len(grouped_ansatze[0])}', '\n')

for idx, group in enumerate(grouped_ansatze):
    unique_ansatze = len(set([tuple(ansatz.operator_order) for ansatz in group]))
    print(f'Group: {idx+1}    Unique Ansatze: {unique_ansatze}')
    for ansatz in group:
        print(f'Result ID: {ansatz.result_id}    Operator Order: {ansatz.operator_order}')
    print()

Total ansatze number: 80
Unique ansatze number: 66
Largest group size: 3 

Group: 1    Unique Ansatze: 2
Result ID: 3.4    Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 7, 10, 2, 5, 12, 11]
Result ID: 3.5    Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 10, 7, 2, 5, 12, 11]
Result ID: 3.9    Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 7, 10, 2, 5, 12, 11]

Group: 2    Unique Ansatze: 1
Result ID: 4.1    Operator Order: [9, 2, 12, 5, 10, 3, 9, 3, 12, 11, 12, 5, 9, 12]
Result ID: 4.2    Operator Order: [9, 2, 12, 5, 10, 3, 9, 3, 12, 11, 12, 5, 9, 12]
Result ID: 4.3    Operator Order: [9, 2, 12, 5, 10, 3, 9, 3, 12, 11, 12, 5, 9, 12]

Group: 3    Unique Ansatze: 3
Result ID: 8.6    Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 8, 9, 10, 7]
Result ID: 8.8    Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 8, 7, 9, 10]
Result ID: 8.10    Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 9, 8, 7, 10]

Group: 4    Unique Ansatze: 1
Result ID: 1.4    Operator Order: [9, 6, 10, 1, 6, 10, 9, 5, 2, 7,

In [7]:
ansatz1 = Ansatz(result_id='5.7')
ansatz2 = Ansatz(result_id='5.10')
s = [round(i, 2) for i in list(np.array(ansatz1.phases_radians) / np.array(ansatz2.phases_radians))]

In [9]:
ansatz1 = Ansatz(result_id='5.8')
ansatz2 = Ansatz(result_id='7.7')

s = [round(i, 2) for i in list(np.array(ansatz1.phases_radians) / np.array(ansatz2.phases_radians))]

In [10]:
for group in grouped_ansatze:
    if len(set([tuple(ansatz.operator_order) for ansatz in group])) > 1:
        for item in [f'Result ID: {ansatz.result_id}  Operator Order: {ansatz.operator_order}' for ansatz in group]:
            print(item)
        print()

Result ID: 3.4  Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 7, 10, 2, 5, 12, 11]
Result ID: 3.5  Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 10, 7, 2, 5, 12, 11]
Result ID: 3.9  Operator Order: [9, 6, 10, 6, 1, 11, 2, 5, 7, 10, 2, 5, 12, 11]

Result ID: 8.6  Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 8, 9, 10, 7]
Result ID: 8.8  Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 8, 7, 9, 10]
Result ID: 8.10  Operator Order: [9, 1, 6, 10, 1, 6, 9, 5, 2, 12, 9, 8, 7, 10]

Result ID: 4.4  Operator Order: [4, 3, 10, 12, 9, 12, 3, 7, 2, 5, 7, 11, 12, 5]
Result ID: 6.7  Operator Order: [4, 3, 10, 12, 9, 12, 3, 7, 2, 5, 12, 11, 7, 5]

Result ID: 5.7  Operator Order: [9, 1, 10, 6, 1, 5, 11, 9, 2, 12, 11, 12, 5, 9]
Result ID: 5.10  Operator Order: [9, 1, 10, 6, 1, 5, 9, 11, 2, 12, 11, 12, 5, 9]

Result ID: 6.9  Operator Order: [9, 6, 9, 7, 9, 8, 1, 5, 2, 11, 12, 11, 12, 5]
Result ID: 6.10  Operator Order: [9, 6, 9, 7, 8, 9, 1, 5, 2, 11, 12, 11, 12, 5]



In [8]:
gadget1 = Gadget(num_qubits=8, phase=1/2, pauli_str='YZZZX')
gadget2 = Gadget(num_qubits=8, phase=-1/2, pauli_str='XZZZY')
gadget3 = Gadget(num_qubits=8, phase=1/2, pauli_str='IYZZZX')
gadget4 = Gadget(num_qubits=8, phase=-1/2, pauli_str='IXZZZY')

gadget = gadget2 + gadget4 + gadget1 + gadget3
print(gadget2.hub_node.ref, gadget4.hub_node.ref, gadget3.hub_node.ref, gadget1.hub_node.ref)
gadget.draw()

WARNING Cannot add <class 'zxfermion.types.Gadget'> and <class 'zxfermion.types.Gadget'>


TypeError: unsupported operand type(s) for +: 'NoneType' and 'Gadget'

In [24]:
from enum import Enum

class LegType(str, Enum):
    X = 'X'
    Y = 'Y'
    Z = 'Z'

s = LegType('Y')